# Nantes: The West Coast of France

## Table of Contents

1. Introduction/Business Problem
2. Data
3. Methodology
4. Results
5. Discussion
6. Conclusion

## An analysis of the neighbourhoods in the primary city of Nantes (excluding suburbs) with special emphasis on schools and top residential areas.

## 1. Introduction/Business Problem

  Tucked in the northwest of France is the thriving city of Nantes with close to 1 million habitants (including population in the suburbs). It is located close to the beaches envelopping the Atlantic Ocean where activities such as surfing, sailing, and kayaking are especially popular. In recent years, Nantes has been experiencing a growing economy due to the increase in industrial activities by large enterprises such as Airbus (airplane manufacturer), Alstom (producer of railway components), and Eurofin (biotechnology company) as well as smaller technology-related startups and companies. For this reason, Nantes has one of the lowest unemployment rates (6%) as compared to the rest of France (around 9%). All of these positive points in addition to the temperate weather makes Nantes a great city to live. However, despite the relatively high standard of living in this city, the crime rate has recently faced an upturn due to the increase in alcohol and illicit drug consumption.

  Since Nantes is in France, there is a paucity of information in English for new or would-be immigrants to Nantes about the different neighbourhoods, especially pertaining to early education and desirable residential areas, as well as areas to avoid. Navigation of the sites in French is difficult (even for those who speak French fluently such as myself) and to the best of my knowledge, there aren't any truly informative data on living in Nantes in English. Thus, in this report, I will examine in detail the different neighbourhoods in the metropole of Nantes (excluding the suburbs) to find schools, types of venues, and best locations to live. For expats living in France, the location of bilingual or international schools as well as residential areas that are close to transportation, shopping, etc. would be especially helpful. This report will be a useful guide to those who will or wish to move or have recently moved with their family to Nantes to start a new life, especially those who are not yet fluent in French.

Objective: To help new or would-be immigrants, who are not yet fluent in French, to find suitable schools for their children and accomodation in Nantes.
This report will provide information on :  
(1) types of venues in each neighbourhood,
(2) schools (elementary, high school, nursery, and bilingual/international),    
(3) average property prices of each neighbourhood, 
(4) areas to avoid,  and  
(5) top areas to live.

## 2. Data

In this report, I will use Wikipedia as well as other online sources printed in French in combination with Foursquare location data to examine the different neighbourhoods (quartiers) in Nantes. Using these information, I will find (1) detailed information such as venues of each neighbourhood, (2) schools (elementary, high school, nursery, and bilingual/international), (3) areas to avoid due to the high level of crime and other types of illicit/illegal activities, (4) average property prices (in € per m^2) of each neighbourhood, and (6) top areas to live. All of these data will be superimposed on a map of Nantes with the neighbourhoods clearly labelled. Depending on the lifestyle of the those who wish to move to Nantes, the type of venues and access to public transportation in the neighbourhoods would be useful to know when choosing a place to live. Especially for expats who wish to put their children in schools that have English programs, it would be very useful to know where the international or bilingual schools are located. I will also include other important aspects of residential planning, such as neighbourhoods that have a bad reputation and average prices in each neighbourhood. This latter point will probably not be found by the Foursquare localization app, so other sources (ie. official documents from the goverment or real estate agencies) will be used to find this information and then it will added to the other data. Lastly, I will give recommendations on the top areas to live based on the data that I have gathered. These data will be very helpful for new/prospective immigrants to Nantes, especially those who do not understand French well.

## 3. Methodology

In this analysis, I used Beautiful Soup to webscrape information off of html webpages, such as Wikipedia. I also downloaded a pdf file containing information on areas in Nantes that have a high crime rate as well as illicit/illegal activities from the Academie de Nantes, which is part of the Ministry of Education, Youth, and Sports that governs formal education and provides youth support. I used tabula to read the pdf into a pandas dataframe. I also downloaded a pdf file (Barometre de Prix, version April 2021) from the official Loire-Atlantique (the prefecture to which Nantes belongs) Notary Public website that contains recent information on the home prices in Nantes. However, the latter was not readable by tabula or other libraries in python so I had to convert the pdf into a csv to read the table about Nantes home prices into a dataframe.
To find the geographical coordinates of the neighbourhoods and other venues, I used geocoder. However, I found that geocoder sometimes gave the wrong coordinates due to other venues with the same or similar name. Thus, I had to run the geocoder sometimes with more precise or slightly modified names. Using these coordinates given by geocoder, I used folium to create maps.
In order to search for and explore venues in the neighbourhoods of Nantes, I used the Foursquare app. I was able to find the top 10 venues in the 11 neighbourhoods of Nantes as well as the different types of schools (elementary, high school, nursery, and bilingual/international) in these neighbourhoods.
Finally, in consideration of the home prices, top 10 venues, schools, and population density, I made recommendations of the most suitable places to live in Nantes.

## 4. Results

In [1]:
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import geocoder
import folium

print('Libraries imported.')

Libraries imported.


### Webscraping information about the neighbourhoods of Nantes from Wikipedia

In order to get information about the neighbourhoods of Nantes, I searched online and found that the most reliable and easily accessible webpage was from Wikipedia. The following is a Wikipedia page that lists the 11 neighbourhoods in the city of Nantes (excluding suburbs).

In [2]:
url = 'https://fr.wikipedia.org/wiki/Liste_des_quartiers_de_Nantes'

I used Beautiful Soup to scrape the name and population number of the 11 neighbourhoods of Nantes.

In [3]:
data  = requests.get(url).text 
soup = BS(data, 'html5lib')

The table with the neighbourhood information had some footnotes and citations included which were specified by the 'sup' and 'span' tags in the table so I had to remove then first before I could get a clean table.

In [4]:
table = soup.find('table')
for sup in table.find_all('sup'):
    sup.unwrap()
for span in table.find_all('span'):
    span.unwrap()
for a in table.find_all('a'):
    a.unwrap()

Then I put the data into a list and subsequently a pandas dataframe.

In [5]:
neighbourhoods = []
for row in table.find_all("tr"):
    if len(row.find_all('td')) >= 1:
        col = row.find_all('td')
        if (col != []):
            Neighbourhood = col[0].text
            Population = col[1].text
        neighbourhoods.append([Neighbourhood, Population])
neighbourhoods

[['Centre-ville[5]', '28 485'],
 ['Bellevue - Chantenay - Sainte-Anne[6]', '25 000'],
 ['Dervallières - Zola[7]', '35 000'],
 ['Hauts-Pavés - Saint-Félix[8]', '35 800'],
 ['Malakoff - Saint-Donatien[9]', '34 669'],
 ['Île de Nantes[10]', '15 818'],
 ['Breil - Barberie[11]', '24 418'],
 ['Nantes Nord[12]', '24 833'],
 ['Nantes Erdre[13]', '26 738'],
 ['Doulon - Bottière[14]', '30 147'],
 ['Nantes Sud[15]', '10 532']]

In [6]:
neighbourhoods0 = pd.DataFrame(neighbourhoods, columns = ['Neighbourhood', 'Population'])

In [7]:
neighbourhoods0

,Neighbourhood,Population
0,Centre-ville[5],28 485
1,Bellevue - Chantenay - Sainte-Anne[6],25 000
2,Dervallières - Zola[7],35 000
3,Hauts-Pavés - Saint-Félix[8],35 800
4,Malakoff - Saint-Donatien[9],34 669
5,Île de Nantes[10],15 818
6,Breil - Barberie[11],24 418
7,Nantes Nord[12],24 833
8,Nantes Erdre[13],26 738
9,Doulon - Bottière[14],30 147


However, although the 'sup' and 'span' tags were removed, the citation marks denoted as [#] were still present so I used the purify function in the regex library to remove them.

In [8]:
import re
def purify(txt):
    return re.sub("\[.+\]", "", txt)

neighbourhoods0['Neighbourhood'] = neighbourhoods0['Neighbourhood'].apply(purify).copy()

In [9]:
neighbourhoods1 = neighbourhoods0.sort_values(by=['Neighbourhood'], ascending=True).reset_index(drop=True)

In [10]:
neighbourhoods1

,Neighbourhood,Population
0,Bellevue - Chantenay - Sainte-Anne,25 000
1,Breil - Barberie,24 418
2,Centre-ville,28 485
3,Dervallières - Zola,35 000
4,Doulon - Bottière,30 147
5,Hauts-Pavés - Saint-Félix,35 800
6,Malakoff - Saint-Donatien,34 669
7,Nantes Erdre,26 738
8,Nantes Nord,24 833
9,Nantes Sud,10 532


The end result was a clean table showing all 11 neighbourhoods of Nantes with their corresponding population. This dataframe will be used for future processing.

### Explore neighbourhoods in Nantes

Using the names of the neighbourhoods of Nantes that I scraped off of the Wikipedia page, I looped them through geocoder to find their corresponding geographical coordinates expressed as latitude and longitude.

In [11]:
from geopy.geocoders import Nominatim

In [12]:
quartiers = neighbourhoods1['Neighbourhood']

quartier_coord = []
for quartier in quartiers:
    address = '{}, Nantes, France'.format(quartier)
    geolocator = Nominatim(user_agent="nantes_explorer_2")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    quartier_coord.append([quartier, latitude, longitude])
    df0 = pd.DataFrame(quartier_coord, columns = ['Neighbourhood', 'Latitude', 'Longitude'])
df0

,Neighbourhood,Latitude,Longitude
0,Bellevue - Chantenay - Sainte-Anne,47.197783,-1.597948
1,Breil - Barberie,47.235155,-1.573885
2,Centre-ville,47.214840,-1.557937
3,Dervallières - Zola,47.217789,-1.588958
4,Doulon - Bottière,47.239410,-1.509438
5,Hauts-Pavés - Saint-Félix,47.228729,-1.564404
6,Malakoff - Saint-Donatien,47.223279,-1.536068
7,Nantes Erdre,47.264979,-1.521609
8,Nantes Nord,47.258410,-1.566323
9,Nantes Sud,47.192114,-1.532469


In [13]:
df = df0.sort_values(by=['Neighbourhood'], ascending=True).reset_index(drop=True)

This dataframe will be used for future analysis.

Then I used these coordinates to construct a map of Nantes using folium.

In [14]:
map_nantes_1= folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, neighbourhood in zip(df["Latitude"], df["Longitude"], df["Neighbourhood"]):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nantes_1)  
    
map_nantes_1

The above is a map showing the foci of the neighbourhoods of Nantes.

In [15]:
CLIENT_ID = 'D2JOFLEXDHMNPZGNMNIJR53L0ILI0UELERUPUB3XL4WREDS1'
CLIENT_SECRET = '0WFKC0ECIIPQ1C1M5OOCHONJBH1NZBTACNHDKCZT5KOUH0ES'
VERSION = '20180605'
LIMIT = 100

Now, I will explore each neighbourhood of Nantes to see what kind of venues are the most common using the Foursquare app. I first explored and obtained the venues within a 1000 meter radius around each neighbourhood and saved them to a dataframe.

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
  
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
nantes_venues = getNearbyVenues(names = df['Neighbourhood'], 
                                latitudes = df['Latitude'], 
                                longitudes = df['Longitude'])

In [18]:
nantes_venues.head()

,Neighbourhood,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bellevue - Chantenay - Sainte-Anne,Parc de La Boucardière,47.201078,-1.597781,Park
1,Bellevue - Chantenay - Sainte-Anne,Gare SNCF de Chantenay,47.197688,-1.593759,Train Station
2,Bellevue - Chantenay - Sainte-Anne,Place Jean Macé,47.199651,-1.589233,Plaza
3,Bellevue - Chantenay - Sainte-Anne,L'Olympic,47.200028,-1.589241,Concert Hall
4,Bellevue - Chantenay - Sainte-Anne,Le Chantenay,47.199609,-1.588970,French Restaurant


Then I put each venue category into a column and determined the average number of each venue in each neighbourhood.

In [19]:
nantes_onehot = pd.get_dummies(nantes_venues[['Venue Category']], prefix="", prefix_sep="")

nantes_onehot['Neighbourhood'] = nantes_venues['Neighbourhood'] 

fixed_columns = [nantes_onehot.columns[-1]] + list(nantes_onehot.columns[:-1])
nantes_onehot = nantes_onehot[fixed_columns]

nantes_onehot.shape

(303, 92)

The app returned 300 venues and 90 different venue categories that were found in the different neighbourhoods. To simplify the dataframe, the venues were grouped by neighbourhoods and the average number of each venue in each neighbourhood is shown.

In [20]:
nantes_grouped = nantes_onehot.groupby('Neighbourhood').mean().reset_index()
nantes_grouped

,Neighbourhood,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Basketball Stadium,Beer Bar,Bike Rental / Bike Share,Bistro,Botanical Garden,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Café,Canal,Canal Lock,Castle,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dessert Shop,Diner,Electronics Store,Factory,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Fountain,French Restaurant,Garden,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Harbor / Marina,Historic Site,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Kebab Restaurant,Lounge,Market,Movie Theater,Newsstand,Opera House,Other Repair Shop,Park,Pedestrian Plaza,Pharmacy,Pizza Place,Playground,Plaza,Pub,Restaurant,Roof Deck,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soccer Stadium,Stadium,Supermarket,Sushi Restaurant,Tea Room,Tourist Information Center,Track Stadium,Train Station,Tram Station,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Bellevue - Chantenay - Sainte-Anne,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.125000,0.00000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.125000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000,0.000000,0.00000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000,0.125000,0.000000,0.125,0.000000,0.000000,0.125000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.125,0.125000,0.000000,0.00,0.000000,0.125000,0.000000
1,Breil - Barberie,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.250000,0.125000,0.000,0.000000,0.00000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000,0.125000,0.000000,0.000,0.125000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.125000,0.125,0.00,0.000000,0.000000,0.000,0.125000,0.000000,0.00,0.000000,0.000000,0.000000
2,Centre-ville,0.01000,0.02,0.010000,0.010000,0.120000,0.000000,0.00000,0.000000,0.020000,0.00000,0.01,0.000000,0.01000,0.000000,0.010000,0.000000,0.000000,0.01,0.01,0.01,0.000000,0.01,0.020000,0.01,0.010000,0.00000,0.000000,0.01,0.000000,0.01,0.010000,0.000000,0.000000,0.020000,0.01,0.00000,0.000000,0.000000,0.100000,0.000000,0.010000,0.01,0.01,0.000000,0.010000,0.000,0.020000,0.00000,0.030000,0.01,0.03,0.01,0.010000,0.020000,0.000000,0.000000,0.0000,0.01,0.000,0.020000,0.060000,0.000,0.020000,0.000000,0.130000,0.020000,0.030000,0.01,0.010000,0.01,0.010000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000,0.01,0.010000,0.000000,0.000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000
3,Dervallières - Zola,0.00000,0.00,0.000000,0.071429,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00,0.000000,0.00000,0.071429,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.071429,0.071429,0.000,0.000000,0.00000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.071429,0.0000,0.00,0.000,0.142857,0.000000,0.000,0.000000,0.000000,0.071429,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.071429,0.000000,0.000000,0.071429,0.00,0.000000,0.000000,0.000

The following shows the top 5 venue categories and their frequency in each neighbourhood.

In [21]:
num_top_venues = 5

for hood in nantes_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = nantes_grouped[nantes_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bellevue - Chantenay - Sainte-Anne----
           venue  freq
0   Concert Hall  0.12
1      Wine Shop  0.12
2          Plaza  0.12
3   Tram Station  0.12
4  Train Station  0.12


----Breil - Barberie----
           venue  freq
0  Grocery Store  0.25
1    Supermarket  0.12
2            Gym  0.12
3   Tram Station  0.12
4    Pizza Place  0.12


----Centre-ville----
               venue  freq
0              Plaza  0.13
1                Bar  0.12
2  French Restaurant  0.10
3   Pedestrian Plaza  0.06
4  Indian Restaurant  0.03


----Dervallières - Zola----
                venue  freq
0                Park  0.14
1            Bus Stop  0.07
2       Track Stadium  0.07
3  Seafood Restaurant  0.07
4          Skate Park  0.07


----Doulon - Bottière----
               venue  freq
0             Bakery  0.25
1              Hotel  0.17
2       Tram Station  0.17
3             Market  0.08
4  French Restaurant  0.08


----Hauts-Pavés - Saint-Félix----
               venue  freq
0             Bake

Lastly, I searched for the 10 most common venue categories in each neighbourhood which were put into a dataframe for further analysis.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = nantes_grouped['Neighbourhood']

for ind in np.arange(nantes_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nantes_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bellevue - Chantenay - Sainte-Anne,Plaza,Wine Shop,French Restaurant,Tram Station,Train Station,Concert Hall,Pharmacy,Park,Dessert Shop,Comedy Club
1,Breil - Barberie,Grocery Store,Gym,Pizza Place,Tram Station,Park,Sushi Restaurant,Supermarket,Yoga Studio,Diner,Concert Hall
2,Centre-ville,Plaza,Bar,French Restaurant,Pedestrian Plaza,Restaurant,Indian Restaurant,Hotel,Bistro,Pizza Place,Pub
3,Dervallières - Zola,Park,Grocery Store,Skate Park,Movie Theater,Supermarket,Plaza,Seafood Restaurant,Gym,Track Stadium,Bakery
4,Doulon - Bottière,Bakery,Tram Station,Hotel,Yoga Studio,French Restaurant,Supermarket,Plaza,Market,Electronics Store,Concert Hall


In [24]:
neighbourhoods_venues_sorted1 = neighbourhoods_venues_sorted.sort_values(by=['Neighbourhood'], ascending=True).reset_index(drop=True)

### Search for schools in Nantes

After obtaining the types of venues that are in each Nantes neighbourhood, I wanted to know the type and number of schools in each area. Thus, I first searched for elementary schools in each neighbourhood of Nantes using Foursquare.

In [25]:
def getNearbySchools(names, latitudes, longitudes, radius=1000, limit=100):

    ES_list= []
    for name, lat, lng in zip(df["Neighbourhood"], df["Latitude"], df["Longitude"]):
 
        categoryId = '4f4533804b9074f6e4fb0105'
         
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat,
            lng,
            categoryId,
            radius, 
            LIMIT)
       
        ES_results = requests.get(url).json()["response"]["venues"]
        
        for data in zip(ES_results):
            for d in data:
                E_name = d['name']
                E_lat = d['location']['lat']
                E_lng = d['location']['lng']
            ES_list.append([(name, E_name, E_lat, E_lng)])
    ES2 = pd.DataFrame([item for E_list in ES_list for item in E_list])
    ES2.columns = ['Neighbourhood','School Name', 'School Latitude', 'School Longitude']
    
    return(ES2)

In [26]:
ES = getNearbySchools(names=df['Neighbourhood'],
                      latitudes=df['Latitude'],
                      longitudes=df['Longitude'])

In [27]:
ES.shape

(26, 4)

In [28]:
ES['School Type'] = 'Elementary'

In [29]:
ES.head()

,Neighbourhood,School Name,School Latitude,School Longitude,School Type
0,Bellevue - Chantenay - Sainte-Anne,École primaire publique Les Reformes - Nantes,47.202412,-1.603280,Elementary
1,Bellevue - Chantenay - Sainte-Anne,Ecole élémentaire Mutualité,47.205500,-1.589398,Elementary
2,Bellevue - Chantenay - Sainte-Anne,Ecole primaire Alain Fournier,47.203730,-1.600433,Elementary
3,Breil - Barberie,École primaire publique Charles Lebourg,47.239204,-1.561498,Elementary
4,Breil - Barberie,La Perverie,47.239627,-1.561995,Elementary


I then grouped the data by neighbourhoods and counted the number of elementary schools in each neighbourhood.

In [30]:
ES_count = ES.groupby('Neighbourhood').count()
ES_count.drop(['School Name', 'School Latitude', 'School Longitude'], axis=1, inplace=True)
ES_count.rename(columns={'School Type':'Elementary'}, inplace=True)
ES_count.reset_index(inplace=True)

I found 26 venues that were labeled as elementary schools in all neighbourhoods of Nantes. Then, I used folium to construct a map that showed the location of each school.

In [31]:
ES_nantes= folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, school in zip(ES["School Latitude"], ES["School Longitude"], ES["School Name"]):
    label = '{}'.format(school)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='crimson',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(ES_nantes)  
    
ES_nantes

Next, I searched for high schools in the different neighbourhoods of Nantes as described above.

In [32]:
def getNearbySchools(names, latitudes, longitudes, radius=1000, limit=100):

    HS_list= []
    for name, lat, lng in zip(df["Neighbourhood"], df["Latitude"], df["Longitude"]):
 
        categoryId = '4bf58dd8d48988d13d941735'
         
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat,
            lng,
            categoryId,
            radius, 
            LIMIT)
       
        HS_results = requests.get(url).json()["response"]["venues"]
        
        for data in zip(HS_results):
            for d in data:
                H_name = d['name']
                H_lat = d['location']['lat']
                H_lng = d['location']['lng']
            HS_list.append([(name, H_name, H_lat, H_lng)])
    HS2 = pd.DataFrame([item for H_list in HS_list for item in H_list])
    HS2.columns = ['Neighbourhood','School Name', 'School Latitude', 'School Longitude']
    
    return(HS2)

In [33]:
HS = getNearbySchools(names=df['Neighbourhood'],
                      latitudes=df['Latitude'],
                      longitudes=df['Longitude'])

In [34]:
HS.shape

(27, 4)

In [35]:
HS['School Type'] = 'High School'

In [36]:
HS.head()

,Neighbourhood,School Name,School Latitude,School Longitude,School Type
0,Breil - Barberie,La Perverie,47.239627,-1.561995,High School
1,Breil - Barberie,Lycee Le Loquidy,47.236376,-1.557094,High School
2,Centre-ville,ENSEC,47.212921,-1.558674,High School
3,Centre-ville,Externat des Enfants Nantais,47.215971,-1.572713,High School
4,Centre-ville,Lycée Gabriel Guist'Hau,47.216336,-1.568325,High School


I then grouped the data by neighbourhoods and counted the number of high schools in each neighbourhood.

In [37]:
HS_count = HS.groupby('Neighbourhood').count()
HS_count.drop(['School Name', 'School Latitude', 'School Longitude'], axis=1, inplace=True)
HS_count.rename(columns={'School Type':'High School'}, inplace=True)
HS_count.reset_index(inplace=True)
HS_count

,Neighbourhood,High School
0,Breil - Barberie,2
1,Centre-ville,9
2,Dervallières - Zola,1
3,Doulon - Bottière,1
4,Hauts-Pavés - Saint-Félix,5
5,Malakoff - Saint-Donatien,2
6,Nantes Erdre,1
7,Nantes Nord,2
8,Nantes Sud,1
9,Île de Nantes,3


I found 27 venues that were labeled as high schools in all neighbourhoods of Nantes. Then, I used folium to construct a map that showed the location of each school.

In [38]:
HS_nantes= folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, school in zip(HS["School Latitude"], HS["School Longitude"], HS["School Name"]):
    label = '{}'.format(school)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(HS_nantes)  
    
HS_nantes

Then, I searched for nursery schools as per previously indicated.

In [39]:
def getNearbySchools(names, latitudes, longitudes, radius=1000, limit=100):

    NS_list= []
    for name, lat, lng in zip(df["Neighbourhood"], df["Latitude"], df["Longitude"]):
 
        categoryId = '4f4533814b9074f6e4fb0107'
         
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat,
            lng,
            categoryId,
            radius, 
            LIMIT)
       
        NS_results = requests.get(url).json()["response"]["venues"]
        
        for data in zip(NS_results):
            for d in data:
                N_name = d['name']
                N_lat = d['location']['lat']
                N_lng = d['location']['lng']
            NS_list.append([(name, N_name, N_lat, N_lng)])
    NS2 = pd.DataFrame([item for N_list in NS_list for item in N_list])
    NS2.columns = ['Neighbourhood','School Name', 'School Latitude', 'School Longitude']
    
    return(NS2)

In [40]:
NS = getNearbySchools(names=df['Neighbourhood'],
                      latitudes=df['Latitude'],
                      longitudes=df['Longitude'])

In [41]:
NS.shape

(15, 4)

In [42]:
NS['School Type'] = 'Nursery'
NS.head()

,Neighbourhood,School Name,School Latitude,School Longitude,School Type
0,Breil - Barberie,La Perverie,47.239627,-1.561995,Nursery
1,Breil - Barberie,Ecole Georges Lafont,47.237393,-1.582578,Nursery
2,Breil - Barberie,Crèche Les Petits Pieds,47.237790,-1.583866,Nursery
3,Centre-ville,Ecole Joseph Blanchard,47.212376,-1.564768,Nursery
4,Centre-ville,École Publique maternelle Frédureau,47.220960,-1.561459,Nursery


I then grouped the data by neighbourhoods and counted the number of nursery schools in each neighbourhood.

In [43]:
NS_count = NS.groupby('Neighbourhood').count()
NS_count.drop(['School Name', 'School Latitude', 'School Longitude'], axis=1, inplace=True)
NS_count.rename(columns={'School Type':'Nursery'}, inplace=True)
NS_count.reset_index(inplace=True)
NS_count

,Neighbourhood,Nursery
0,Breil - Barberie,3
1,Centre-ville,3
2,Dervallières - Zola,1
3,Hauts-Pavés - Saint-Félix,4
4,Malakoff - Saint-Donatien,2
5,Nantes Sud,2


I found 15 venues that were labeled as nursery schools in all neighbourhoods of Nantes. Then, I used folium to construct a map that showed the location of each school.

In [44]:
NS_nantes= folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, school in zip(NS["School Latitude"], NS["School Longitude"], NS["School Name"]):
    label = '{}'.format(school)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(NS_nantes)  
    
NS_nantes

Finally, I searched for international and/or bilingual schools where instruction is in English part or most of the time using Foursquare. However, Foursquare was not able to give me information on these types of schools even with query or categoryId. Thus, I had to search online and came up with the following list of international and/or bilingual schools in Nantes. Then I used geocoder to find the geographical coordinates of each school.

In [45]:
bilingual_schools = ['Sainte Madeleine','Ecole Saint Michel','Ecole Montessori de Nantes','Ecole elementaire Anatole de Monzie','College Briand','Lycee Mandela','International School of Nantes']

BS_list = []
for school in bilingual_schools:
    address = '{}, Nantes, France'.format(school)
    geolocator = Nominatim(user_agent="nantes_explorer_1")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print(school, latitude, longitude)
    BS_list.append([school, latitude, longitude])
    BS = pd.DataFrame(BS_list, columns = ['School Name', 'School Latitude', 'School Longitude'])

Sainte Madeleine 47.2029752 -1.5737022
Ecole Saint Michel 47.21511845 -1.5702468749922927
Ecole Montessori de Nantes 47.2171231 -1.5495297
Ecole elementaire Anatole de Monzie 47.20285225 -1.5416172947901479
College Briand 47.24925265 -1.4947872170179148
Lycee Mandela 47.2083198 -1.533540256174522
International School of Nantes 47.210929 -1.613601


In [46]:
BS['School Type'] = 'Bilingual'
BS.head()

,School Name,School Latitude,School Longitude,School Type
0,Sainte Madeleine,47.202975,-1.573702,Bilingual
1,Ecole Saint Michel,47.215118,-1.570247,Bilingual
2,Ecole Montessori de Nantes,47.217123,-1.549530,Bilingual
3,Ecole elementaire Anatole de Monzie,47.202852,-1.541617,Bilingual
4,College Briand,47.249253,-1.494787,Bilingual


However, the associated neighbourhoods in which the schools are located were not indicated by geocoder. Thus, to find the neighbourhood to which they belonged, I constructed a map using folium that showed both bilingual/international schools (denoted as black circles) and the neighbourhoods (denoted as blue circles). 

In [47]:
BS_nantes= folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, school in zip(BS["School Latitude"], BS["School Longitude"], BS["School Name"]):
    label = '{}'.format(school)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        label=BS["School Name"],
        parse_html=False).add_to(BS_nantes)
for lat, long, neighbourhood in zip(df["Latitude"], df["Longitude"], df["Neighbourhood"]):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        label=df["Neighbourhood"],
        parse_html=False).add_to(BS_nantes) 
    
BS_nantes

The map above shows the international/bilingual schools in relation to the neighbourhood foci as to determine in which neighbourhood the school is located. The blue circle that was closest to the school was considered as the associated neighbourhood in which the school belonged. I manually made a list of the associated neighbourhoods and added them into the BS dataframe as shown below.

In [48]:
BS['Neighbourhood'] = ['Île de Nantes', 'Centre-ville', 'Centre-ville', 'Île de Nantes', 'Doulon - Bottière', 'Île de Nantes', 'Bellevue - Chantenay - Sainte-Anne']

I then grouped the data by neighbourhoods and counted the number of bilingual/international schools in each neighbourhood.

In [49]:
BS_count = BS.groupby('Neighbourhood').count()
BS_count.drop(['School Name', 'School Latitude', 'School Longitude'], axis=1, inplace=True)
BS_count.rename(columns={'School Type':'Bilingual'}, inplace=True)
BS_count.reset_index(inplace=True)
BS_count

,Neighbourhood,Bilingual
0,Bellevue - Chantenay - Sainte-Anne,1
1,Centre-ville,2
2,Doulon - Bottière,1
3,Île de Nantes,3


In [50]:
frames = [ES, HS, NS, BS]
nantes_schools_list = pd.concat(frames)
nantes_schools_list

,Neighbourhood,School Name,School Latitude,School Longitude,School Type
0,Bellevue - Chantenay - Sainte-Anne,École primaire publique Les Reformes - Nantes,47.202412,-1.603280,Elementary
1,Bellevue - Chantenay - Sainte-Anne,Ecole élémentaire Mutualité,47.205500,-1.589398,Elementary
2,Bellevue - Chantenay - Sainte-Anne,Ecole primaire Alain Fournier,47.203730,-1.600433,Elementary
3,Breil - Barberie,École primaire publique Charles Lebourg,47.239204,-1.561498,Elementary
4,Breil - Barberie,La Perverie,47.239627,-1.561995,Elementary
5,Breil - Barberie,École maternelle Jacques Prévert,47.231064,-1.577158,Elementary
6,Breil - Barberie,École Villa Maria,47.230804,-1.562521,Elementary
7,Centre-ville,Ecole Elementaire Saint Nicolas,47.216205,-1.562842,Elementary
8,Centre-ville,Ecole Emilie Pehant,47.211717,-1.549089,Elementary
9,Centre-ville,Ecole Primaire Jean Jaures,47.219422,-1.559645,Elementary


Then I merged all the school count dataframes into one big school dataframe that contains the neighbourhoods with the corresponding count of Elementary, High, Nursery, and Bilingual/International Schools in order to determine the number of different types of schools in each neighbourhood.

In [51]:
nantes_school1 = ES_count.merge(HS_count, how='outer', on='Neighbourhood')

In [52]:
nantes_school2 = nantes_school1.merge(NS_count, how='outer', on='Neighbourhood')

In [53]:
nantes_schools3 = nantes_school2.merge(BS_count, how='outer', on='Neighbourhood')
nantes_schools4 = nantes_schools3.replace(np.nan,0)
nantes_schools = nantes_schools4.sort_values(by=['Neighbourhood'], ascending=True).reset_index(drop=True)

In [54]:
nantes_schools

,Neighbourhood,Elementary,High School,Nursery,Bilingual
0,Bellevue - Chantenay - Sainte-Anne,3.0,0.0,0.0,1.0
1,Breil - Barberie,4.0,2.0,3.0,0.0
2,Centre-ville,4.0,9.0,3.0,2.0
3,Dervallières - Zola,2.0,1.0,1.0,0.0
4,Doulon - Bottière,0.0,1.0,0.0,1.0
5,Hauts-Pavés - Saint-Félix,5.0,5.0,4.0,0.0
6,Malakoff - Saint-Donatien,2.0,2.0,2.0,0.0
7,Nantes Erdre,1.0,1.0,0.0,0.0
8,Nantes Nord,0.0,2.0,0.0,0.0
9,Nantes Sud,3.0,1.0,2.0,0.0


Then I merged all the school count dataframes in order to determine the number of different types of schools in each neighbourhood.  This dataframe will be used for future analysis.

### Real estate prices in Nantes by neighbourhood

After considering the types of venues and schools in each neighbourhood, another important aspect of living in a new city is the cost of homes. Thus, I searched online for the most recent and accurate data on home prices in Nantes. I found a pdf file called 'Barometre de Prix version Avril 2021' (released in April of this year), which lists the average real estate prices in Nantes, from the official site of the Loire-Atlantique (the prefecture in which Nantes is located) Notary Public Association. In France, notary publics are the agents that preside over the real estate transactions and they regularly release information on real estate prices.
The tables from this pdf file were not readable using any python library, so I converted the pdf into a csv file and then used pandas to read into a dataframe.

In [55]:
prix = pd.read_csv('/Users/caroline/Desktop/BAROMETRE_PRIX_AVRIL_2021.csv')

In [56]:
prix.head()

,Quartier,Neuf,Ancien,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,NANTES,4 670€,3 650 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Centre-ville,6 150€,4 380 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chantenay - Bellevue - Sainte-Anne,4 750€,3 450 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dervallières - Zola,5 230€,3 840 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hauts Pavés - Saint Félix,6 150€,4 070 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since the table contained many additional columns that were not useful, I only saved the 3 columns with data.

In [57]:
prix_df = prix[['Quartier', 'Neuf', 'Ancien']]

In [58]:
prix_df

,Quartier,Neuf,Ancien
0,NANTES,4 670€,3 650 €
1,Centre-ville,6 150€,4 380 €
2,Chantenay - Bellevue - Sainte-Anne,4 750€,3 450 €
3,Dervallières - Zola,5 230€,3 840 €
4,Hauts Pavés - Saint Félix,6 150€,4 070 €
5,Saint Donatien - Malakoff,4 950€,3 770 €
6,Ile de Nantes,5 040€,3 390 €
7,Breil - Barberie,4 780 €,3 230 €
8,Quartiers Nord,4 720€,3 090 €
9,Nantes Erdre,3 980 €,3 270 €


Since the column headings were in French, I renamed them into English and clarified the units.

In [59]:
prix_df1 = prix_df.rename(columns={'Quartier': 'Neighbourhood', 'Neuf': 'New Homes (per m^2)', 'Ancien': 'Old Homes (per m^2)'})
prix_df1

,Neighbourhood,New Homes (per m^2),Old Homes (per m^2)
0,NANTES,4 670€,3 650 €
1,Centre-ville,6 150€,4 380 €
2,Chantenay - Bellevue - Sainte-Anne,4 750€,3 450 €
3,Dervallières - Zola,5 230€,3 840 €
4,Hauts Pavés - Saint Félix,6 150€,4 070 €
5,Saint Donatien - Malakoff,4 950€,3 770 €
6,Ile de Nantes,5 040€,3 390 €
7,Breil - Barberie,4 780 €,3 230 €
8,Quartiers Nord,4 720€,3 090 €
9,Nantes Erdre,3 980 €,3 270 €


Due to the presence of accents in the French language, I had to change the neighbourhood names in order to synchronize them with those in other dataframes to facilitate cross-referencing.

In [60]:
prix_df1.loc[0,['Neighbourhood']] = 'Nantes'
prix_df1.loc[2,['Neighbourhood']] = 'Bellevue - Chantenay - Sainte-Anne'
prix_df1.loc[4,['Neighbourhood']] = 'Hauts-Pavés - Saint-Félix'
prix_df1.loc[5,['Neighbourhood']] = 'Malakoff - Saint-Donatien'
prix_df1.loc[6,['Neighbourhood']] = 'Île de Nantes'
prix_df1.loc[8,['Neighbourhood']] = 'Nantes Nord'
prix_df1.loc[11,['Neighbourhood']] = 'Nantes Sud'
prix_df1

,Neighbourhood,New Homes (per m^2),Old Homes (per m^2)
0,Nantes,4 670€,3 650 €
1,Centre-ville,6 150€,4 380 €
2,Bellevue - Chantenay - Sainte-Anne,4 750€,3 450 €
3,Dervallières - Zola,5 230€,3 840 €
4,Hauts-Pavés - Saint-Félix,6 150€,4 070 €
5,Malakoff - Saint-Donatien,4 950€,3 770 €
6,Île de Nantes,5 040€,3 390 €
7,Breil - Barberie,4 780 €,3 230 €
8,Nantes Nord,4 720€,3 090 €
9,Nantes Erdre,3 980 €,3 270 €


The above table shows the average prices of brand new and used homes defined as € per m^2 in each neighbourhood of Nantes as well as the city as a whole.

Next, I merged the above dataframe with the previous dataframes that showed the neighbourhood population (df) and the geographical coordinates (neighbourhoods1. But first I also had to change the names a bit to ensure the names were the same in all 3 dataframes.

In [61]:
df2 = pd.merge(df, neighbourhoods1)

I also added a row called 'Nantes' with the coordinates of the city itself and it's population (according to wikipedia). I had to add this manually because the values in the Population column were not parsable strings so I couldn't use pd.to_numeric() and df.sum() functions.

In [62]:
df2.loc[-1] = ['Nantes', 47.2186371, -1.5541362, 303382].copy()
df2.index = df2.index + 1
df2 = df2.sort_index()
df2

,Neighbourhood,Latitude,Longitude,Population
0,Nantes,47.218637,-1.554136,303382
1,Bellevue - Chantenay - Sainte-Anne,47.197783,-1.597948,25 000
2,Breil - Barberie,47.235155,-1.573885,24 418
3,Centre-ville,47.214840,-1.557937,28 485
4,Dervallières - Zola,47.217789,-1.588958,35 000
5,Doulon - Bottière,47.239410,-1.509438,30 147
6,Hauts-Pavés - Saint-Félix,47.228729,-1.564404,35 800
7,Malakoff - Saint-Donatien,47.223279,-1.536068,34 669
8,Nantes Erdre,47.264979,-1.521609,26 738
9,Nantes Nord,47.258410,-1.566323,24 833


In [63]:
nantes_info = pd.merge(df2, prix_df1)

In [64]:
nantes_info

,Neighbourhood,Latitude,Longitude,Population,New Homes (per m^2),Old Homes (per m^2)
0,Nantes,47.218637,-1.554136,303382,4 670€,3 650 €
1,Bellevue - Chantenay - Sainte-Anne,47.197783,-1.597948,25 000,4 750€,3 450 €
2,Breil - Barberie,47.235155,-1.573885,24 418,4 780 €,3 230 €
3,Centre-ville,47.214840,-1.557937,28 485,6 150€,4 380 €
4,Dervallières - Zola,47.217789,-1.588958,35 000,5 230€,3 840 €
5,Doulon - Bottière,47.239410,-1.509438,30 147,3 780€,2 960 €
6,Hauts-Pavés - Saint-Félix,47.228729,-1.564404,35 800,6 150€,4 070 €
7,Malakoff - Saint-Donatien,47.223279,-1.536068,34 669,4 950€,3 770 €
8,Nantes Erdre,47.264979,-1.521609,26 738,3 980 €,3 270 €
9,Nantes Nord,47.258410,-1.566323,24 833,4 720€,3 090 €


This dataframe summarizes the average prices of homes, geographical coordinates, and population in the different neighbourhood of Nantes as well as Nantes as a whole.  This dataframe will be used for future analysis.

### Explore neighbourhoods to avoid in Nantes

Another very important aspect to consider when choosing a neighbourhood in a new city to live in is safety and security. Although Nantes is a city that is economically stable with low unemployment rate, there are some problems with crime. There is quite a high rate of theft/robbery (60.43%) followed by personal violence (12.51%). In Nantes, as in many other cities, the latter is mostly linked to neighbourhoods that are considered not so safe due to a high level of illicit/illegal activities, such as prostitution, drug use, alcohol abuse, etc. The government of France regularly issues a list of areas that are "les quartiers prioritaires de la politique de la ville" (QPV), which are considered as "urban renewal projects", "urban red zones", "urban no-go zones", or "inner-city projects" in English. Thus, I obtained data that showed neighbourhoods to avoid in Nantes.
I downloaded an official copy of the "LISTE DES ZONES URBAINES SENSIBLES" which was released by the Academie de Nantes, the organization in Nantes that governs education and youth support. I then used tabula to read the pdf into a pandas dataframe. Since tabula output is actually a list of dataframes, I chose the first one (avoid[0]) that contains the list of QPV in Nantes.

In [65]:
import tabula
import os

In [66]:
avoid = tabula.read_pdf("Liste ZUS.pdf", pages="all")

In [67]:
avoid2 = pd.DataFrame(avoid[0])

In [68]:
avoid2.head()

,COMMUNE,QUARTIER
0,NaN,LOIRE-ATLANTIQUE (44)
1,Nantes,Les Dervallières
2,Nantes,Malakoff
3,Nantes,Quartier Est
4,Nantes,Quartiers Nord


In [69]:
avoid3 = avoid2.iloc[1:6]

I isolated the section for Nantes which gave 5 neighbourhoods. Due to the accents and difference in names (some sources call Nantes Nord as Quartier Nord and Bottiere as Quartier Est), I had to rename some of the neighbourhoods.

In [70]:
avoid3.columns = ['City', 'Neighbourhood']
avoid3.at[1, 'Neighbourhood'] = 'Dervallières'
avoid3.at[3, 'Neighbourhood'] = 'Bottière'
avoid3.at[4, 'Neighbourhood'] = 'Nantes Nord'

In [71]:
avoid3

,City,Neighbourhood
1,Nantes,Dervallières
2,Nantes,Malakoff
3,Nantes,Bottière
4,Nantes,Nantes Nord
5,Nantes/Saint Herblain,Bellevue


Then I used geocoder to find the coordinates of these areas to avoid.

In [72]:
areas_avoid = avoid3['Neighbourhood']

aa_list = []
for area in areas_avoid:
    address = '{}, Nantes, France'.format(area)
    geolocator = Nominatim(user_agent="nantes_explorer_1")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print(area, latitude, longitude)
    aa_list.append([area, latitude, longitude])
    aa = pd.DataFrame(aa_list, columns = ['Area', 'Latitude', 'Longitude'])
aa

Dervallières 47.2237199 -1.5976517
Malakoff 47.2152706 -1.5264284
Bottière 47.2392597 -1.5196349
Nantes Nord 47.258410350000005 -1.5663227883277482
Bellevue 47.1124609 -1.28523


,Area,Latitude,Longitude
0,Dervallières,47.223720,-1.597652
1,Malakoff,47.215271,-1.526428
2,Bottière,47.239260,-1.519635
3,Nantes Nord,47.258410,-1.566323
4,Bellevue,47.112461,-1.285230


I had to search again for Bellevue because there seems to be a village near Nantes called Bellevue and the geocoder gave the wrong coordinates. So I had to manually change the coordinates of Bellevue to those of the neighbourhood in Nantes.

In [73]:
geolocator = Nominatim(user_agent="nantes_explorer_1")
location = geolocator.geocode('bellevue/chantenay, nantes, france')
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

47.2077116 -1.6060371


In [74]:
aa.at[4, 'Latitude'] = 47.2077116
aa.at[4, 'Longitude'] = -1.6060371

In [75]:
aa

,Area,Latitude,Longitude
0,Dervallières,47.223720,-1.597652
1,Malakoff,47.215271,-1.526428
2,Bottière,47.239260,-1.519635
3,Nantes Nord,47.258410,-1.566323
4,Bellevue,47.207712,-1.606037


In [76]:
aa_nantes= folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, area in zip(aa["Latitude"], aa["Longitude"], aa["Area"]):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='brown',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(aa_nantes)
aa_nantes

## 5. Discussion

### Analysis for the best neighbourhoods to live in Nantes

After obtaining all the necessary data, I analyzed all the accumulated information to find the best neighbourhoods to live in Nantes. A definition of a nice neighbourhood to reside in largely depends on the preferences and lifestyle of the residents. For a family with small children, it would be ideal to live in a neighbourhood that is medium-priced, not very dense with easy access to parks, shops, and public transportation. Further, proximity to schools and a safe environment are also very important. If the residents are a young couple or single, proximity to entertainment venues, restaurants, and coffee shops may be attractive attributes in a neighbourhood. In any case, neighbourhoods that are considered as QPVs may not be sought after. Thus, I removed the 5 neighbourhoods that are considered as QPVs (Bellevue, Dervallieres, Malakoff, Bottiere, and Nantes Nord) from our dataframe.

In [77]:
nantes_info

,Neighbourhood,Latitude,Longitude,Population,New Homes (per m^2),Old Homes (per m^2)
0,Nantes,47.218637,-1.554136,303382,4 670€,3 650 €
1,Bellevue - Chantenay - Sainte-Anne,47.197783,-1.597948,25 000,4 750€,3 450 €
2,Breil - Barberie,47.235155,-1.573885,24 418,4 780 €,3 230 €
3,Centre-ville,47.214840,-1.557937,28 485,6 150€,4 380 €
4,Dervallières - Zola,47.217789,-1.588958,35 000,5 230€,3 840 €
5,Doulon - Bottière,47.239410,-1.509438,30 147,3 780€,2 960 €
6,Hauts-Pavés - Saint-Félix,47.228729,-1.564404,35 800,6 150€,4 070 €
7,Malakoff - Saint-Donatien,47.223279,-1.536068,34 669,4 950€,3 770 €
8,Nantes Erdre,47.264979,-1.521609,26 738,3 980 €,3 270 €
9,Nantes Nord,47.258410,-1.566323,24 833,4 720€,3 090 €


In [78]:
best_neighbourhoods = nantes_info.drop([1,4,5,7,9]).reset_index(drop=True)

In [79]:
best_neighbourhoods

,Neighbourhood,Latitude,Longitude,Population,New Homes (per m^2),Old Homes (per m^2)
0,Nantes,47.218637,-1.554136,303382,4 670€,3 650 €
1,Breil - Barberie,47.235155,-1.573885,24 418,4 780 €,3 230 €
2,Centre-ville,47.214840,-1.557937,28 485,6 150€,4 380 €
3,Hauts-Pavés - Saint-Félix,47.228729,-1.564404,35 800,6 150€,4 070 €
4,Nantes Erdre,47.264979,-1.521609,26 738,3 980 €,3 270 €
5,Nantes Sud,47.192114,-1.532469,10 532,4 470€,3 010 €
6,Île de Nantes,47.207048,-1.546210,15 818,5 040€,3 390 €


From this list of neighbourhoods, we can further break down the attributes that are most desirable for the different types of residents. 
In order to see which neighbourhoods are good to raise a family, I looked at the areas to avoid, house prices, and also what kind of venues are mostly in the neighbourhood. Since Centre-ville and Ile de Nantes seem to have a lot of entertainment venues (bars, restaurants, etc), they may not be so suitable to raise a family, so they were dropped too which left us with 4 neighbourhoods that are suitable for families with kids: Hauts Pave-St. Felix, Breil-Barberie, Nantes Erdre, and Nantes Sud.

In [80]:
best_neighbourhoods2 = best_neighbourhoods.loc[[0,1,3,4,5]].reset_index(drop=True)
best_neighbourhoods2

,Neighbourhood,Latitude,Longitude,Population,New Homes (per m^2),Old Homes (per m^2)
0,Nantes,47.218637,-1.554136,303382,4 670€,3 650 €
1,Breil - Barberie,47.235155,-1.573885,24 418,4 780 €,3 230 €
2,Hauts-Pavés - Saint-Félix,47.228729,-1.564404,35 800,6 150€,4 070 €
3,Nantes Erdre,47.264979,-1.521609,26 738,3 980 €,3 270 €
4,Nantes Sud,47.192114,-1.532469,10 532,4 470€,3 010 €


Then I sliced out these 4 neighbourhoods from the previous dataframe, neighbourhoods_venues_sorted1, to see the top 10 most common venues within these four areas.

In [81]:
neighbourhoods_venues_sorted2 = neighbourhoods_venues_sorted1.loc[[1,5,7,9]]
neighbourhoods_venues_sorted2

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Breil - Barberie,Grocery Store,Gym,Pizza Place,Tram Station,Park,Sushi Restaurant,Supermarket,Yoga Studio,Diner,Concert Hall
5,Hauts-Pavés - Saint-Félix,Tram Station,Bakery,Supermarket,French Restaurant,Pizza Place,Farmers Market,Playground,Plaza,Garden,Restaurant
7,Nantes Erdre,Bus Stop,Harbor / Marina,Hotel,Other Repair Shop,Supermarket,Botanical Garden,Tram Station,Garden,Convenience Store,Cosmetics Shop
9,Nantes Sud,Bus Stop,Park,Tram Station,Hotel,Supermarket,Yoga Studio,Electronics Store,Concert Hall,Convenience Store,Cosmetics Shop


I also sliced out these 4 neighbourhoods from the previous dataframe, nantes_schools, to determine the number of schools within these four areas.

In [82]:
nantes_schools1 = nantes_schools.loc[[1,5,7,9]]
nantes_schools1

,Neighbourhood,Elementary,High School,Nursery,Bilingual
1,Breil - Barberie,4.0,2.0,3.0,0.0
5,Hauts-Pavés - Saint-Félix,5.0,5.0,4.0,0.0
7,Nantes Erdre,1.0,1.0,0.0,0.0
9,Nantes Sud,3.0,1.0,2.0,0.0


In consideration of the home prices and types of venues in these 4 neighbourhoods, we would notice that Hauts Pave-St. Felix appears to be quite an affluent neighbourhood that is close to public transportation, shops, restaurants, and playground, which would be a highly desirable residential area for wealthy families. Further, there is a high number of elementary, high, and nursery schools in this neighbourhood which would be convenient for taking the kids to school. However, the population is relatively high in this area so may be quite dense and crowded. This suggests that the living quarters may be small for a high price.
For middle income families, I would recommend Breil-Barberie in consideration of the home prices, which are below average as compared to the average home price in Nantes as a whole (best_neighbourhoods2[0]), and the types of venues. The latter includes easy access to public transportation, shops, and parks, which makes it ideal for average income families. Further, the population is also not as high as in the Hauts Pave-St. Felix area, which may be explained by the presence of more parks in the area. On top of that, there are also many different schools in this neighbourhood as compared to Nantes Erdre which has a similar home price and venue profile, but with a lot less schools. I think that Nantes Sud may also be less desirable because the population is so low as compared to the other areas (about 2.5-3 times lower) and the home prices are quite a bit below average (especially for old homes) suggesting that this neighbourhood may not be as developed as the other areas. However, for those who prefer living closer to the country-side, this neighbourhood would be ideal for them.
In contrast, there are no bilingual/international schools in these areas, but if those schools are desired, the easy access to public transportation in both neighbourhoods renders the commute to school easier.

On the other hand, if the residents are young couples or single, then it may be more attractive for them to live where entertainment venues are closer, such as in Centre-ville and Île de Nantes. Thus, I constructed other dataframes containing information on the home prices, population, venues, and schools in those areas.

In [83]:
best_neighbourhoods3 = best_neighbourhoods.loc[[0,2,6]].reset_index(drop=True)
best_neighbourhoods3

,Neighbourhood,Latitude,Longitude,Population,New Homes (per m^2),Old Homes (per m^2)
0,Nantes,47.218637,-1.554136,303382,4 670€,3 650 €
1,Centre-ville,47.214840,-1.557937,28 485,6 150€,4 380 €
2,Île de Nantes,47.207048,-1.546210,15 818,5 040€,3 390 €


In [84]:
neighbourhoods_venues_sorted3 = neighbourhoods_venues_sorted1.loc[[2,10]]
neighbourhoods_venues_sorted3

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Centre-ville,Plaza,Bar,French Restaurant,Pedestrian Plaza,Restaurant,Indian Restaurant,Hotel,Bistro,Pizza Place,Pub
10,Île de Nantes,French Restaurant,Lounge,Bar,Hotel,Plaza,Sandwich Place,Brewery,Pedestrian Plaza,Pizza Place,Historic Site


In [85]:
nantes_schools2 = nantes_schools.loc[[2,10]]
nantes_schools2

,Neighbourhood,Elementary,High School,Nursery,Bilingual
2,Centre-ville,4.0,9.0,3.0,2.0
10,Île de Nantes,2.0,3.0,0.0,3.0


As indicated, the home prices in Centre-ville, which is considered as the downtown of Nantes, are considerably higher than average Nantes prices, which suggests that this area is very affluent. Centre-ville is full of shopping centres bars, and restaurants, for those who prefer easy access to entertainment and socializing. Interestingly, there is a very high number of schools in this area, as well as bilingual/international schools, so this neighbourhood may be suitable for affluent expat families who are indifferent to the lack of play facilities for children and the high presence of entertainment venues.
On the other hand, the most common venues in the Île de Nantes are restaurants, bars, and hotels, which suggests that this neighbourhood is a rather touristic area. The old home prices are considerably lower and the new home prices are much higher than average Nantes prices which suggests that there is a dichotomy of this neighbourhood. The population in this area is also quite a bit lower than in the other areas (except Nantes Sud) without the presence of parks (which take up space) suggesting that there are fewer residents and more venues in this neighbourhood.

In [86]:
best_neighbourhoods.to_csv('best_neighbourhoods.csv')
best_neighbourhoods2.to_csv('best_neighbourhoods2.csv')
best_neighbourhoods3.to_csv('best_neighbourhoods3.csv')
neighbourhoods_venues_sorted2.to_csv('neighbourhoods_venues_sorted2.csv')
neighbourhoods_venues_sorted3.to_csv('neighbourhoods_venues_sorted3.csv')
nantes_schools1.to_csv('nantes_schools1.csv')
nantes_schools2.to_csv('nantes_schools2.csv')

## 6. Conclusion

In conclusion, my analysis showed that there are 6 out of 11 neighbourhoods that are relatively safer to live in Nantes. For affluent families, Haut Paves-St. Felix and Centre-ville have the highest home prices, but also most number of schools. The latter also has 2 bilingual/international schools. Haut Paves-St. Felix is more suitable for residents who prefer a more kids-oriented environment, while Centre-ville has more entertainment venues and very few parks/playgrounds. For middle-income families, I recommend Breil-Barberie which also has a good number of schools, the home prices are average, the population is also average, and the top 3 venues are grocery store, park, and tram stop. Thus, this neighbourhood would be great for families with kids since it isn't very dense and very convenient to shop, play, and commute. These points make the neighbourhood of Breil-Barberie a good place for a middle-income family to settle.